In [1]:
%matplotlib qt5

import pymodels
import pyaccel
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
# parametros de courant snyder a partir de uma nova optica
from mathphys.functions import save_pickle, load_pickle

model = pymodels.si.create_accelerator()
model.cavity_on = True
model.radiation_on = True

# To calculate things that we need it is necessary to define the attributes in this way
# it is necessary to keep in mind that to calculate the variation of the beam stay clear
# linear model this attributes must be turned into False

In [2]:
h_pos = pyaccel.lattice.get_attribute(model, 'hmax', indices='closed')
h_neg = pyaccel.lattice.get_attribute(model, 'hmin', indices='closed')

### beam stay clear calculation for betatron oscilations

In [3]:
def calc_fbscx(acc, h_pos, h_neg):
    spos = pyaccel.lattice.find_spos(acc, indices='closed')
    twi,*_ = pyaccel.optics.calc_twiss(acc, indices='closed')
    betax = twi.betax
    
    a_sup = h_pos / np.sqrt(betax)
    a_inf = h_neg / np.sqrt(betax)
    
    a_def = np.minimum(a_sup, -a_inf)
    a_st = np.min(a_def)
    
    BSC = a_st * np.sqrt(betax)

    index = np.argmin(np.abs(BSC - h_pos))
#     bsc graphic with lattice 
    fig, ax = plt.subplots()
    
    ax.plot(spos, BSC, color='blue', label='BSC')
    ax.plot(spos, -BSC, color='blue')
    
    ax.plot(spos, h_pos, color='black', label='vchamber')
    ax.plot(spos, h_neg, color='black')
    
    ax.scatter(spos[index], BSC[index], color='purple')
    
    ax.set_xlabel('s [m]')
    ax.set_ylabel('amplitude [$m^{1/2}$]')
    
    pyaccel.graphics.draw_lattice(model, height=0.0009, offset=0, gca=True)
    
    ax.legend()
    
    plt.tight_layout()
    plt.show()    
    

In [5]:
calc_fbscx(model, h_pos,h_neg)

### Beam stay clear for betatron oscilations plus new non-linear orbit

In [12]:
# Cálculo do BSC a partir de uma órbita fechada (tracking)
def calc_sbsc(acc, offp):
    acc.cavity_on=False
    acc.radiation_on=False
    acc.vchamber_on=False

    twi,*_ = pyaccel.optics.calc_twiss(acc, energy_offset=offp, indices='open')
    twin,*_ = pyaccel.optics.calc_twiss(acc, energy_offset=-offp, indices='open')

    betax, rx = twi.betax, twi.rx
    betaxn, rxn = twin.betax, twin.rx
    etax = twi.etax

    h_pos= pyaccel.lattice.get_attribute(acc, 'hmax', indices='open')
    h_neg= pyaccel.lattice.get_attribute(acc, 'hmin', indices='open')

    a_max = (np.minimum(h_pos-rx , np.abs(h_neg-rx)))/np.sqrt(betax)
    idx_min = np.argmin(a_max)
    a_def = a_max[idx_min]

    BSC_sup = a_def * np.sqrt(betax) + rx
    BSC_inf = -a_def * np.sqrt(betaxn) + rxn

    spos = pyaccel.lattice.find_spos(acc)

    # Plot new beam stay clear com tracking

    plt.figure()
    plt.plot(spos, BSC_sup, 'b',label='$BSC (\delta = {})$'.format(offp)) #plot do beam stat clear superior dependente de um desvio de 
                                                    #energia
    plt.plot(spos, BSC_inf, 'b')
    plt.plot(spos, h_pos, 'k', label='câmara de vácuo') #plot da camara de vacuo
    plt.plot(spos, h_neg, 'k')

    # plt.plot(spos, calcularc_orb(offp, 'xc'), 'c', label ='$x_c (\delta)$')

    plt.plot(spos, etax*offp, label='nova órbita')
    
    plt.xlabel('s [m]')
    plt.ylabel('amplitude [m]')

    plt.xlim(0,125)
    pyaccel.graphics.draw_lattice(acc, offset=0, height=0.0009, gca= True)
    plt.legend()
    plt.show()

In [13]:
calc_sbsc(model, offp=0.03)

### Versão otimizada para o cálculo das amplitudes (abertura física)

In [6]:
def calc_amp(acc,energy_offsets, hmax, hmin):
    a_def = np.zeros(energy_offsets.size)
    indices = np.zeros(energy_offsets.size)
    try:
        for idx,delta in enumerate(energy_offsets):
            twi,*_ = pyaccel.optics.calc_twiss(accelerator=acc,
                                              energy_offset=delta,
                                              indices='closed')
            if np.any(np.isnan(twi[0].betax)):
                raise pyaccel.optics.OpticsException('error')
            rx = twi.rx
            betax = twi.betax

            a_sup = (hmax - rx)**2 /betax
            a_inf = (hmin - rx)**2 /betax

            a_max = np.minimum(a_sup,a_inf)
            idx_min = np.argmin(a_max)
            indices[idx] = idx_min
            a_def[idx] = a_max[idx_min]
    except (pyaccel.optics.OpticsException, pyaccel.tracking.TrackingException):
        pass
    return np.sqrt(a_def), indices

def calc_amp_y(acc,energy_offsets, vmax, vmin):
    a_def = np.zeros(energy_offsets.size)
    indices = np.zeros(energy_offsets.size)
    try:
        for idx,delta in enumerate(energy_offsets):
            twi,*_ = pyaccel.optics.calc_twiss(accelerator=acc,
                                              energy_offset=delta,
                                              indices='closed')
            if np.any(np.isnan(twi[0].betay)):
                raise pyaccel.optics.OpticsException('error')
            ry = twi.ry
            betay = twi.betay

            a_sup = (vmax - ry)**2 /betay
            a_inf = (vmin - ry)**2 /betay

            a_max = np.minimum(a_sup,a_inf)
            idx_min = np.argmin(a_max)
            indices[idx] = idx_min
            a_def[idx] = a_max[idx_min]
    except (pyaccel.optics.OpticsException, pyaccel.tracking.TrackingException):
        pass
    return np.sqrt(a_def), indices

# cálculo do novo curly_H a partir das novas funções ópticas
# ---------------------------------
def curl_H(desvio, model):
    
    twi,*_ = pyaccel.optics.calc_twiss(accelerator=model, energy_offset= desvio)
    
    rx = twi.rx
    px = twi.px

    betax_dev, betay_dev = twiss1.betax, twiss1.betay
    alphax_dev, alphay_dev = twiss1.alphax, twiss1.alphay
    gammax_dev, gammay_dev = twiss1.gammax, twiss1.gammay
    
    H_cur = gammax_dev*rx**2 + 2*alphax_dev*rx*px + betax_dev*px**2
    
    return np.sqrt(H_cur)

### Where electrons are lost (tracking)

In [7]:
# Função criada em outro programa utilizando tracking

# ---------------------
def track_eletrons(deltas, n_turn, element_idx, model, pos_x=1e-5, pos_y=3e-6):

    #     pos_x and pos_y have default values but it can be varied
    #     here we define the initial conditions for the simulation
    
    orb = pyaccel.tracking.find_orbit6(model, indices=[0, element_idx])
    orb = orb[:, 1]
    
    rin = np.zeros((6, deltas.size))
    rin += orb[:, None]
    rin[0] += pos_x
    rin[2] += pos_y
    rin[4] += deltas
    
    track = pyaccel.tracking.ring_pass(
        model, rin, nr_turns=n_turn, turn_by_turn=True,
        element_offset=element_idx, parallel=True)
    
    par_out, flag, turn_lost, element_lost, plane_lost = track
    
    turnl_element = []

    for i,item in enumerate(turn_lost):
        if item == n_turn and element_lost[i] == element_idx: # ignora elétrons que não foram perdidos
            pass
        else:
            turnl_element.append((item, element_lost[i], deltas[i]))
    
    return turnl_element

## Where the electrons are lost along the ring (all of s positions)

In [8]:
# This function will calculate where the electrons are lost using tracking

# The function recives an array containing all the s positions along the ring 

def trackm_elec(deltas, n_turn, lspos):
    results = []
    ind = []
    spos = pyaccel.lattice.find_spos(model)
    
    for k, iten in enumerate(lspos):
        
        el_idx = np.argmin(np.abs(spos-iten)) # selecting the index to calculate the lost positions
        turnl = track_eletrons(deltas, n_turn, el_idx, model, pos_x=1e-5, pos_y=3e-6)
        results.append(turnl) # 
        ind.append(el_idx)
        
    return results, ind


# function returns the desired element index of dipoles, quadrupoles, sextupoles and any other 
# element that is desired, the only thing that is needed is to pass a string 

def el_idx_collector(acc, lname):
    all_index = []
    fam_data = pymodels.si.get_family_data(acc)

    for string in lname:
        element_index = []
        array_idx = np.array(fam_data[string]['index'], dtype=object)

        for k, lista in enumerate(array_idx):
            length = len(lista)

            if length % 2 != 0:
                ind = int((length-1)/2)
                element_index.append(lista[ind])

            else:
                try:
                    ind = int(length/2 + 1)
                    element_index.append(lista[ind])
                
                except IndexError:
                    ind = int(length/2 - 1)
                    element_index.append(lista[ind])
                    
        all_index.append(element_index)
    
    return all_index

### Function that plots (BSC x Energy Deviation) and the lost electrons after tracking

- This function also plots the magnetic lattice and beta function when radiation and cavity are on

In [9]:
accepn, accepp = pyaccel.optics.calc_touschek_energy_acceptance(model)

In [10]:
spos = pyaccel.lattice.find_spos(model, indices='open')

In [11]:

def plot_track(lista_resul, lista_idx, lista_off, param, element_idx, accep):
    # ----------------
    
    m = pymodels.si.create_accelerator()
    m.cavity_on = False
    m.radiation_on= False
    m.vchamber_on = False
    twi0,*_ = pyaccel.optics.calc_twiss(m,indices='open')
    betax = twi0.betax
    betax = betax*(1/5)
    
    fig, (a1, a2) = plt.subplots(1, 2, figsize=(10, 5), sharey=True, gridspec_kw={'width_ratios': [1, 3]})
    
    a1.grid(True, alpha=0.5, ls='--', color='k')
    a1.tick_params(axis='both', labelsize=12)
    a2.grid(True, alpha=0.5, ls='--', color='k')
    a2.tick_params(axis='both', labelsize=12)
    a2.xaxis.grid(False)
    a1.xaxis.grid(False)

    a1.set_title(r'$\delta \times$ lost turn', fontsize=16)
    if 'pos' in param:
        for item in lista_resul:
            a1.plot(item[0], item[2]*1e2, 'k.', label = '')
            a1.set_ylabel(r'positive $\delta$ [%]', fontsize=14)
    
    elif 'neg' in param:
        for item in lista_resul:
            a1.plot(item[0], -item[2]*1e2, 'k.', label = '')
            a1.set_ylabel(r'negative $\delta$ [%]', fontsize=14)
            
    a1.set_xlabel(r'n de voltas', fontsize=14)
    

    a2.set_title(r'tracking ', fontsize=16)

    # Tracking graphics
    # ----------------
    
#     defining the acceptance given a point s of the ring

    acp_s = accep[element_idx]
    ind = np.argmin(np.abs(lista_off-acp_s))
    a2.plot(spos[lista_idx][:ind], lista_off[:ind]*1e2,'b.', label=r'accep. limit', alpha=0.25)
    
    if 'pos' in param:
        a2.plot(spos[int(lista_resul[1][-1])], lista_resul[2][-1]*1e2, 'r.', label='lost pos. (track)')
        for item in lista_resul:
            a2.plot(spos[int(item[1])], item[2]*1e2, 'r.')
            
    elif 'neg' in param:
        a2.plot(spos[int(lista_resul[1][-1])], -lista_resul[2][-1]*1e2, 'r.', label='lost pos. (track)')
        for item in lista_resul:
            a2.plot(spos[int(item[1])], -item[2]*1e2, 'r.')
            
#     plotting acceptances with plt.hlines
    
    plt.hlines(1e2*acp_s, spos[0], spos[-1], color='black', linestyles='dashed', alpha=0.5)
    
    # linear model graphic
    # ----------------     
    
    # plotting beta function 
    a2.plot(spos, np.sqrt(betax),color='orange', label=r'$ \sqrt{\beta_x}  $')
        
    # plotting magnetic lattice
    pyaccel.graphics.draw_lattice(model, offset=-0.5, height=0.5, gca=True)

    # initial position that tracking begins
    
    a2.plot(spos[element_idx], 0, 'ko', label='{}, ({} m)'.format(
        model[element_idx].fam_name, "%.2f" % spos[element_idx]))
    
    # setting configurations of the graphic
    a2.set_xlabel(r'$s$ [m]', fontsize=14)
    a2.legend(loc='best', ncol=2)

    fig.tight_layout()
    fig.show()

## linear model

In [12]:
energy_offsets = np.linspace(0,0.046,460)

model.cavity_on = False
model.radiation_on = False
model.vchamber_on = False

### Positive amplitutes

In [13]:
amplitudes_pos = calc_amp(model, energy_offsets, h_pos, h_neg)
amplitude_pos, idx_limitante_pos = amplitudes_pos 
idx_limitante_pos = np.array(idx_limitante_pos, dtype = int)

In [13]:
amplitude_pos

array([0.00239933, 0.00239624, 0.00239315, 0.00239006, 0.00238697,
       0.00238388, 0.00238078, 0.00237769, 0.0023746 , 0.00237151,
       0.00236842, 0.00236533, 0.00236223, 0.00235914, 0.00235605,
       0.00235296, 0.00234987, 0.00234677, 0.00234368, 0.00234059,
       0.00233749, 0.0023344 , 0.00233131, 0.00232821, 0.00232512,
       0.00232203, 0.00231893, 0.00231584, 0.00231274, 0.00230965,
       0.00230656, 0.00230346, 0.00230037, 0.00229727, 0.00229418,
       0.00229108, 0.00228799, 0.00228489, 0.00228179, 0.0022787 ,
       0.0022756 , 0.00227251, 0.00226941, 0.00226631, 0.00226322,
       0.00226012, 0.00225702, 0.00225393, 0.00225083, 0.00224773,
       0.00224464, 0.00224154, 0.00223844, 0.00223534, 0.00223225,
       0.00222915, 0.00222605, 0.00222295, 0.00221985, 0.00221676,
       0.00221366, 0.00221056, 0.00220746, 0.00220436, 0.00220126,
       0.00219816, 0.00219506, 0.00219196, 0.00218886, 0.00218576,
       0.00218266, 0.00217956, 0.00217646, 0.00217336, 0.00217

### Negative amplitutes

In [13]:
amplitudes_neg = calc_amp(model, -energy_offsets, h_pos, h_neg)
amplitude_neg, idx_limitante_neg = amplitudes_neg 
idx_limitante_neg = np.array(idx_limitante_neg, dtype = int)

In [14]:
fig, ax = plt.subplots(figsize=(10,4))

ax.set_title('dependência da amplitude com os desvios de energia', fontsize=14)

ax.plot(energy_offsets[:355], amplitude_pos[:355], color='black', label='postive amp')
ax.plot(-energy_offsets, amplitude_neg, color='blue', label='negative amp')

ax.legend(loc='best')

# plt.tight_layout()

ax.set_xlabel('$\delta$ [%]', fontsize=14)
ax.set_ylabel(r'amplitude [$m^{1/2}$]', fontsize=14)

plt.show()

### Tracking

In [17]:
model.cavity_on = True
model.radiation_on = True
model.vchamber_on = False

spos = pyaccel.lattice.find_spos(model)
lspos = spos[:-1]
deltas = np.linspace(0.001, 0.1, 2)

shape = spos.shape
lind = np.indices(shape)
lind = lind.squeeze()

In [14]:
rs_pos = load_pickle('/home/ABTLUS/thales.bastos/Downloads/rs_posl')

In [ ]:
rs_neg = load_pickle('/home/ABTLUS/thales.bastos/Downloads/rs_negl')

In [ ]:
fam_data = pymodels.si.get_family_data(model)

### Anotações 

- 1 Comportamentos semelhantes, quadrupolos (todos os quadrupolos), dispositivos de inserção, sextupolos SDB0, SDA0, SFA0, SFB0, SFP0, SFP2. Os dipolos centrais (BC) também exibem um padrão semelhante.

- 2 Os sextupolos SDB1, SDB2, SDA1, SDA2, SDA3, SFA1, SFA2, SFB1 exibem comportamentos semelhantes gerando padrões de perda com desvios de energia possíveis, os elétrons acabam se perdendo na previsão dos limitantes físicos

- 3 Os sextupolos SDB3, SFB2, SFP1, geram padrões de perda com desvios de energia um pouco altos, as perdas se deram em grande parte nos limitantes para desvios de energia mais proximos a aceitancia da máquina. Os dipolos B1 e B2 também possuem este comportamento para alguns trechos.

- 4 O dipolo B1 e B2 do trecho 5 mostraram um comportamento diferente, semelhante ao caso 2

### selecting the index of the desired elements along the ring

In [15]:
lname = ['BC','B1','B2','QDB2', 'SDB1']

all_index = el_idx_collector(model, lname)

BC_index = all_index[0]
B2_index = all_index[2]
B1_index = all_index[1]

QDB2_index = all_index[3]
SFP1_index = all_index[4]

In [ ]:
c = SFP1_index[3]
plot_track(rs_pos[c], idx_limitante_pos,energy_offsets,'pos', lind[c], accepp)

In [22]:
spos[1046]

84.29923250000009

In [541]:
c= Q1_index[20]
plot_track(data_neg[c], idx_limitante_neg, energy_offsets,'neg', lindn[c])

### Comentando a respeito dos gráficos obtidos

### $(\delta > 0)$

- Existe uma região onde o tracking bate com o modelo linear;

- Após determinado desvio de energia $\delta _{nl}$ o regime não linear parece dominar a dinâminca;

- A partir deste $\delta _{nl}$ os elétrons, em geral, se perdem na volta, $n= 0$; 

- Observando os $\delta$ 's de energia negativos algo diferente acontece;
---

### $(\delta < 0)$

- O modelo linear implementado para o caso negativo apresenta algumas limitações comparando com as simulações de tracking. Para alguns elementos regiões caóticas são mais comuns

- De maneira análoga ao caso $(\delta > 0)$, existe uma região de caos, onde não se sabe exatamente onde o elétron se perde a cada volta



## Plot das amplitudes

In [84]:
# concatenando os dois arrays para amplitudes e desvios de energia
#--------------
lisa = np.r_[list_a_inf, list_a]
lisoff = np.r_[list_off_neg, list_off]

# Plot
#--------------
plt.figure()
plt.plot(lisoff*1e2, lisa,'b,',label =r'$a_{min}$')
plt.ylabel(r'$a_{min}$')
plt.xlabel(r'$\delta$ [%]')
plt.legend()
plt.show()

# Calculando $\mathscr{H}$

### Dinâmica não linear

- Para calcular as amplitudes geradas por espalhamento Touschek as seguintes relações foram utilizadas 
---
$ J = \frac{1}{2}\bigg( {\gamma x^{2}+2 \alpha x \cdot p_{x} +\beta p_{x}^{2}} \bigg)$

- Como a amplitude é dada em função da ação

$\Rightarrow$ $ a = \sqrt{ 2 J}$ 

- Assim, a amplitude é dada em termos de $\mathscr{H}$

$ a = \sqrt{\gamma x^{2}+2 \alpha x \cdot p_{x} +\beta p_{x}^{2}}$,
$\mathscr{H} = \frac{1}{\beta}\bigg[ {x^{2}+ ({\beta p_{x}+ \alpha x})^{2} } \bigg]$

- Portanto,

$a = \sqrt{\mathscr{H}}$, neste caso $\mathscr{H} = \mathscr{H} _{tracking}(x, p_{x})$

---
### Dinâmica linear
- No caso em que a dinânimca é linear e a nova órbita depende linearmente do desvio de energia, $\mathscr{H}$ assume a seguinte forma;

$\mathscr{H} = \frac{1}{\beta}\bigg[ {\eta^{2}+ ({\beta \eta '+ \alpha \eta})^{2} } \bigg]$ $\Rightarrow$ $\mathscr{H}(\eta, \eta') = \gamma \eta^{2}+2 \alpha \eta \cdot \eta ' +\beta \eta '^{2}$

- Neste caso, as amplitudes são reescritas em termos de um desvio de energia $\delta$

$ a = \delta \sqrt{\gamma \eta^{2}+2 \alpha \eta \cdot \eta ' +\beta \eta '^{2}}$

In [11]:
# Nesta etapa vamos calcular o curly H para as novas funções ópticas e seus nvoso valores serão armazenados em um
# array

c = len(list_off)
off = 0
delta = 0.0001
lista_H = []

for _ in range(c):
    
    H = curl_H(off)
    off += delta
    lista_H.append(H)

In [12]:
c = len(list_off_neg)
off = 0
delta = -0.0001
lista_H_inf = []

for _ in range(c):
    
    H = curl_H(off)
    off += delta
    lista_H_inf.append(H)

In [13]:
model.cavity_on = False
model.radiation_on = False
model.vchamber_on = False

lista_H = np.array(lista_H)
H_transpose = np.transpose(lista_H)

lista_H_inf = np.array(lista_H_inf)
H_transpose_inf = np.transpose(lista_H_inf)

## Plotando os gráficos das amplitudes

In [86]:
# plt.figure()

# plt.subplot(2,1,1)
# plt.title(r'Gráfico $\delta_{pos}$')
# plt.plot(list_off*1e2, list_a, 'k')
# for item in H_transpose:
#     plt.plot(list_off*1e2, item,'b,')

# plt.legend()
# plt.ylabel(r'amplitudes $(a)$')
# plt.xlabel(r'$\delta$ [%]')

# plt.subplot(2,1,2)
# plt.title(r'Gráfico $\delta_{neg}$')
# plt.plot(list_off*1e2, list_a_inf, 'k')
# for item in H_transpose_inf:
#     plt.plot(list_off*1e2, item,'b,')

# plt.tight_layout(h_pad=1)
# plt.legend()
# plt.ylabel(r'amplitudes $(a)$')
# plt.xlabel(r'$\delta$ [%]')
# plt.show()

## Cálculo das aceitancias de energia

In [224]:
plt.figure()
plt.plot(spos, lista_H_inf[84])

In [15]:
accep = []
accep_neg = []

for item in H_transpose:
    dif  = np.abs(item-list_a)
    idx = np.argmin(dif)
    accep.append(list_off[idx])

for item in H_transpose_inf:
    dif  = np.abs(item-list_a_inf)
    idx = np.argmin(dif)
    accep_neg.append(list_off_neg[idx])

accep = np.array(accep)    
accep_neg = np.array(accep_neg)

### Plot das aceitancias 

In [31]:
# # Plotando a aceitancia positiva 
# plt.figure()

# plt.title(r'Gráfico $\delta _{pos}$')
# plt.subplot(1,2,1)

# plt.plot(spos, accep*1e2,'k-', label='aceitancia')
# plt.legend()
# plt.xlabel(r'$s$ [m]')
# plt.ylabel(r'$\delta $ [%]')

# plt.title(r'Gráfico $\delta _{neg}$')
# plt.subplot(1,2,2)

# plt.plot(spos, accep_neg*1e2,'k', label='aceitancia')
# plt.legend()
# plt.xlabel(r'$s$ [m]')
# plt.ylabel(r'$ - \delta $ [%]')

# plt.tight_layout(w_pad=2,rect = (0,0,3,2))
# plt.show()

### Plotando meu modelo e tracking e ao mesmo tempo a função beta para um desvio de energia

### Plotando aceitancias positivas e negativas juntas

In [16]:
# offs = np.r_[list_off_neg, list_off]
# s = np.r_[spos[list_idx_inf],spos[list_idx]]

# plt.figure()

# # plt.plot(spos, accep*1e2, 'b', label = r'$\delta _{pos} (s)$')
# # plt.plot(spos, accep_neg*1e2, 'k', label = r'$\delta _{neg} (s)$')
# plt.plot(s, (offs*1e2),'r.', label=r'limitantes', markersize = 4)
# pyaccel.graphics.draw_lattice(model, offset=0, height=0.1, gca=True)

# plt.xlabel(r'$s$ [m]')
# plt.ylabel(r'$energy \ acceptance \ -\delta(s)$ [%]')
# # plt.xlim(0,100)
# plt.legend()
# plt.show()

### Anotações

- Utilizando o código para o cálculo da aceitancia touschek pode-se perceber:

- As taxas de perda de elétrons nas regiões que possuem dipolos B1, B2 e BC são as que mais proporcionam a perda de elétrons ao longo da trajetória do anel de armazenamento;

- Isso ocorre porque essas são regiões baixo beta e, nesse sentido, a densidade de elétrons é maior;

- A densidade eletrônica é uma grandeza importante, porque quanto mais elétrons constituírem o feixe, maiores são as chances de ocorrer espalhamento Touschek;

- Percebe-se que nos trechos retos de baixo beta não ocorre perda de elétrons por espalhamento Touschek e isso é coerente porque a aceitancia de energia nessas regiões é mais alta.

---
 ### Perguntas:
 
 - Em qual limite de desvio de energia devo trabalhar ?


### Espalhamento touschek

- Ocorre devido às colisões entre os elétrons ou partículas que constituem o feixe

- O fenômeno mais relevante no espalhamento Touschek é a troca de momento transversal e longitudinal;

- É importante ressaltar que uma pequena quantidade de momento transversal gera um grande desvio de momento longitudinal; (https://arxiv.org/pdf/physics/9903034.pdf) $\Rightarrow$ [As anotações a seguir são referentes a este artigo]

- Esta amplificação de momento ocorre devido a efeitos relativísticos, de forma que a variação de momento longitudinal é aumentada pelo fator de Lorentz.

- O espalhamento Touschek é diferente do denominado espalhamento intra-feixe, uma vez que o último é responsável pela variação das dimensões do feixe de elétrons e este efeito também é caracterizado por colisões múltiplas de elétrons. O espalhamento Touschek, por sua vez, é um fenômeno que ocorre devido a colisão de partículas individuais. 

- Esse efeito intra-feixe também é devido ao espalhamento Coulomb.

#### Apenas por curiosidade: 

- O espalhamento Touschek foi vizulizado pela primeira vez em um pequeno acelerador de partículas denominado ADA.

- Isso se tornou um grande problema em aneis de armazenamento e em fonstes de luz sincrotron uma vez que o tempo de vida era drasticamente reduzido

---

- Na maior parte das abordagens associadas aos cálculos de tempode vida apenas a transferência de momento horizontal para longitudinal foi considerado, enquanto que a troca de momento transversal para vertical foi negligenciado;

- É a partir da abordagem deste artigo que passa a se considerar a troca de momento transversal para a vertical e, neste caso, a distribuição bidimensional de momento na transversal levando a uma dependência mais fraca com o desvio máximo de energia;



### Cell for visualizing the lattice with beta function and testing

In [167]:
#This cell was used to vizualize the lattice when I was understandig what locals the touschel scattering
# is most probable analizing the beta function and looking for the lattice
# I think that the future of this cell will be the trash, but I'll keep this for a little while




AttributeError: module 'matplotlib.pyplot' has no attribute 'tyght_layout'

In [92]:
vmax = pyaccel.lattice.get_attribute(model,'vmax',indices='closed')
vmin = pyaccel.lattice.get_attribute(model,'vmin',indices='closed')

In [ ]:
energy_offsets = np.linspace(0,0.046,460)
calc_y = calc_amp_y(model, deltas, vmax, vmin)

In [103]:
amplitudes_pos_y, indices = calc_y

# depois preciso plotar estes gráficos que foram calculados agora para desvios de energia no eixo y 

# preciso saber se a aceitancia em y varia, precios me responder urgentemente essa questão